In [26]:
#Importar bibliotecas do sistemas
import sys
import os

# Caminho absoluto até a pasta src
sys.path.append(os.path.abspath("../src"))

#Importar libs do python
import pandas as pd 
import numpy as np 
import math
from pyautocad import Autocad, APoint 

#Importar libs do programa 
import design_functions
from v_p_chapa_cabeca.v_p_chapa_cabeca import *
from draw_autocad.draw_autocad_figures import *
from materials import * 

In [27]:
perfil_escolhido = W_530x109_0


aço_escolhido = ASTM_A36

solda_escolhida =E70XX

perfil_escolhido.inercias()
perfil_escolhido.material(aço_escolhido)


##### Cálculo da disposição da chapa, dos parafusos e do diâmetro minimo

In [42]:
M = 9806.7 #kN.mm 

V = 9.8067 #kN

T = 9.8067 #kN

parafuso = ASTM_A325
parafuso.prop_geometricas(rosca = 1,planos_de_corte =1)

S = dim_chapa_parafuso(M,V,T,perfil_escolhido,disposicoes_gerdau_chapa_cabeca,parafuso,gamma)


In [29]:
S

[0,
 59.81562515015006,
     parafuso  x (mm)  y (mm)
 0          1      35    68.8
 1          2     105    68.8
 2          3     175    68.8
 3          4     245    68.8
 4          5      35   510.2
 5          6     105   510.2
 6          7     175   510.2
 7          8     245   510.2
 8          9      35   589.0
 9         10     105   589.0
 10        11     175   589.0
 11        12     245   589.0]

In [30]:

chapa = S[3]
ver_parafuso = S[4]

N_parafusos = len(ver_parafuso)  #Número total de parafusos

S[1].diametro_pol

'1/2'

In [31]:
chapa.df

vértice  x (mm)  y (mm)
0        1       0       0
1        2     280       0
2        3     280     619
3        4       0     619
4        5       0       0

##### Cálculo da espessura da chapa de cabeça

In [32]:
rigida = 1

r_parafuso_total = resistencia_total(parafuso,gamma)

s_p_m =solicitante_parafuso_momento(M,chapa.B,ver_parafuso, parafuso , S[0])
s_p_t = solicitante_parafuso_tração(T,N_parafusos)


exp = exp_placa(aço_escolhido, chapa, rigida, ver_parafuso, parafuso.diametro_mm, r_parafuso_total,(s_p_m + s_p_t),gamma)

exp

6.35

##### Cálculo da espessura do filete de solda na alma e na mesa

In [33]:
espessura= espessura_solda(M,V,T,solda_escolhida,perfil_escolhido,exp,True,gamma)

esp = math.ceil(espessura) #mm

esp

5

##### Abertura do projeto no pyautocad

In [34]:
acad = Autocad(create_if_not_exists=True)
acad.prompt("Hello, Autocad from Python\n")
print(acad.doc.Name)

Hello, Autocad from Python

Desenho3.dwg


In [35]:
limpar_desenho(acad)

✅ Desenho limpo com sucesso.


In [36]:
pontos_hexagono = gerar_pontos_hexagono(parafuso.diametro_mm)

In [37]:
# Chamando a função para desenhar a chapa 3D
objetos_chapa = criar_chapa_3d(acad, chapa.df, exp)

In [38]:
objetos_parafusos=[]
for i in range(ver_parafuso.shape[0]):
    x_centro = ver_parafuso.iat[i, 1]
    y_centro = ver_parafuso.iat[i, 2]

    # Adicionar circunferência no ponto
    obj = acad.model.AddCircle(APoint(x_centro, y_centro,exp), parafuso.diametro_mm / 2)
    objetos_parafusos.append(obj)
    obj = acad.model.AddCircle(APoint(x_centro, y_centro,0), parafuso.diametro_mm / 2)
    objetos_parafusos.append(obj)
    # Transladar hexágono para o ponto atual
    hexagono_transladado = transladar_pontos(pontos_hexagono, x_centro, y_centro, exp)

    for j in range(len(hexagono_transladado) - 1):
        p1 = APoint(*hexagono_transladado[j])
        p2 = APoint(*hexagono_transladado[j + 1])
        obj = acad.model.AddLine(p1, p2)
        objetos_parafusos.append(obj)

In [39]:
objetos_secao_perfil = desenhar_secao_perfil(acad, perfil_escolhido, (chapa.B / 2) - (perfil_escolhido.b_f / 2), posicao_y=20, altura_z=exp)

In [40]:
# Rotacionar apenas a seção do perfil:
for obj in objetos_parafusos:
    obj.Rotate3D(APoint(0, 0, 0), APoint(1,0, 0), math.radians(90))
    obj.Rotate3D(APoint(0, 0, 0), APoint(0,0, 1), math.radians(90))

for obj in objetos_chapa:
    obj.Rotate3D(APoint(0, 0, 0), APoint(1,0, 0), math.radians(90))
    obj.Rotate3D(APoint(0, 0, 0), APoint(0,0, 1), math.radians(90))

for obj in objetos_secao_perfil:
    obj.Rotate3D(APoint(0, 0, 0), APoint(1,0, 0), math.radians(90))
    obj.Rotate3D(APoint(0, 0, 0), APoint(0,0, 1), math.radians(90))

# Vetor de translação (exemplo: mover 100 mm no eixo X)
dx, dy, dz = 0,-perfil_escolhido.b_f/2,0  # ajuste aqui conforme necessário

# Aponta o vetor de deslocamento
vetor = APoint(dx, dy, dz)

for obj in objetos_secao_perfil:
    obj.Move(APoint(0,0,0),vetor)


for obj in objetos_chapa:
    obj.Move(APoint(0,0,0),vetor)


for obj in objetos_parafusos:
    obj.Move(APoint(0,0,0),vetor)

In [41]:
escrever_descricao(acad,0,0,max(chapa.df["y (mm)"])+10 ,"Chapa","",perfil_escolhido.nome,esp)